<a href="https://colab.research.google.com/github/SY-256/anomaly_detection/blob/main/notebook/chapter5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 二項分布の最尤推定による異常検知

## モデルの学習

二項分布の発生率（成功確率）パラメータμの推定値$\hat{\mu}$を最尤推定で求める

※「**すべての選手が同じ成功確率**$\mu$**を持つという仮定**（＝過分散なし）」を置くことに相当し、最尤推定で求められる成功確率パラメータ$\hat{\mu}$は、全選手で共通と仮定したスリーポイントシュートの成功確率にあたる。

In [ ]:
# 二項分布の最尤推定による異常検知の実装例（学習）
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#### 学習データの読み込みと前処理
# スリーポイントシュート成功データ
df_train = pd.read_csv("https://raw.githubusercontent.com/ghmagazine/python_anomaly_detection_book/refs/heads/main/notebooks/datasets/player_stats_2022.csv")
df_train = df_train[df_train["FG3A"] > 0] # 試行回数0を除外
x_train = df_train["FG3M"].to_numpy() # 成功回数（発生数）
n_train = df_train["FG3A"].to_numpy() # 試行回数

#### 学習ステップ1. 正常のモデルを作成する ####
mu = np.sum(x_train) / np.sum(n_train) # 学習データ全体での発生率（発生率パラメータμの最尤推定）

#### 学習ステップ2. 異常を表す指標（異常度）を定義する ####
# 式のみで定義

#### 学習ステップ3. 異常度に閾値を設ける ####
# 推論時に都度算出するので、学習時は閾値を設定しない

#### 学習で求めたパラメータを表示 ####
print(f"mu={mu}")

ここで求めた`mu`が、発生率パラメータの最尤推定量$\hat{\mu}$になる。学習データ（正常データ）に関して、試行数$n$を横軸、成功率$\frac{x}{n}$を縦軸に取った散布図をプロット

In [ ]:
# 試行数nと成功率x/nのプロット
import seaborn as sns

# 成功率
mu_train = x_train / n_train
# 描画用のFigureとAxesの生成
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 5))
# 分母nと成功率x/nの関係を描画
sns.scatterplot(x=n_train, y=mu_train, c="#999999", ax=ax)
ax.set_xlabel("n (FG3A)", fontsize=12)
ax.set_ylabel("x/n (3P%)", fontsize=12)
# グラフを表示
plt.show()

試行回数$n$が小さいほど成功率$\frac{x}{n}$のばらつきが大きく、平均から外れた値が観測されやすい。よって成功率$\frac{x}{n}$に閾値を設けてしまうと、$n$が小さい選手ほど誤報が増える。そのため、$n$に合わせて閾値を変化させる必要があり、それを実現するためには発生率パラメータの最尤推定量$\hat{\mu}$と二項分布モデルを用いて、推論時に閾値を動的に決める必要がある。